In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
import mne
import numpy as np
from tqdm import tqdm
import pandas as pd
mne.set_log_level(40)

In [ ]:
df = pd.read_csv('data/participants.tsv', sep='\t')
labels = {'left': 0, 'right': 1}

In [ ]:
def eeg_power_band(epochs):
    FREQ_BANDS = {
        'delta': [0.5, 4],
        'theta': [4, 8],
        'alpha': [8, 12],
        'beta': [12, 30]
    }
    spectrum = epochs.compute_psd(picks='eeg', fmin=0.5, fmax=30.)
    psds, freqs = spectrum.get_data(return_freqs=True)
    psds /= np.sum(psds, axis=-1, keepdims=True)
    X = []
    for fmin, fmax in tqdm(FREQ_BANDS.values()):
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band.reshape(len(psds), -1))
    return np.concatenate(X, axis=1)

In [14]:
X, y = [], []
for f in tqdm(os.listdir('data/epochs')):
    epochs = mne.read_epochs('data/epochs/'+f, preload=True)[1:]
    X.append(epochs)
    for epoch in epochs:
        y.append(labels[df[df['participant_id']==f.replace('.fif', '')]['attended_ch'].values[0]])
y = np.array(y)

100%|██████████| 36/36 [00:05<00:00,  6.04it/s]


In [15]:
X = eeg_power_band(mne.concatenate_epochs(X))

100%|██████████| 4/4 [00:01<00:00,  3.76it/s]


In [16]:
np.save('data/psd/X.npy', X)
np.save('data/psd/y.npy', y)